In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
pd.read_csv("Meet.csv")

,Unnamed: 0,Place,TeamPlace,LastName,FirstName,Year,Team,AvgMile,Time,Score,Meet,1k,2.4k,4.5k,7k,5k,3k,5.8k
0,7,8.0,1,Lepak,Jake,SR-4,Macalester,5:20.6,21:22.1,8.0,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12,13.0,2,Johnson,Adrian,JR-3,Macalester,5:25.4,21:41.4,13.0,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,16.0,3,Hayes,August,FR-1,Macalester,5:30.4,22:01.4,16.0,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,17.0,4,Reyes Herrera,Edwin,SR-4,Macalester,5:31.5,22:05.9,17.0,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,19.0,5,Mayse,Smith,SO-2,Macalester,5:31.7,22:06.7,19.0,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,32,33.0,6,Kerr,Fred,SO-2,Macalester,5:41.2,22:44.7,32.0,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,34,35.0,7,O'Donnell-Hoff,Sam,SR-4,Macalester,5:42.5,22:49.9,34.0,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,44,45.0,8,Milner,Michael,SR-4,Macalester,5:48.2,23:12.5,NaN,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,50,51.0,9,Peske,Cole,FR-1,Macalester,5:52.4,23:29.6,NaN,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,51,52.0,10,Hunsanger,Brett,SO-2,Macalester,5:52.7,23:30.6,NaN,Twin Cities Invitational,NaN,NaN,NaN,NaN,NaN,NaN,NaN
